In "[Literary Pattern Recognition](https://www.journals.uchicago.edu/doi/full/10.1086/684353)", Long and So train a classifier to differentiate haiku poems from non-haiku poems, and find that many features help do so.  In class, we've discussed the importance of representation--how you *describe* a text computationally influences the kinds of things you are able to do with it.  While Long and So explore description in the context of classification, in this homework, you'll see how well you can design features that can differentiate these two classes *without* any supervision. Are you able to featurize a collection of poems such that two clusters (haiku/non-haiku) emerge when using KMeans clustering, with the text representation as your only degree of freedom?

In [34]:
import csv, os, re
import nltk
from scipy import sparse
from sklearn.cluster import KMeans
from sklearn import metrics
import math
from collections import Counter
import random

In [35]:
def read_texts(path, metadata, filepath_col):
    data=[]
    with open(metadata, encoding="utf-8") as file:
        csv_reader = csv.reader(file)
        next(csv_reader)
        for cols in csv_reader:
            poem_path=os.path.join(path, cols[filepath_col])
            if os.path.exists(poem_path):
                with open(poem_path, encoding="utf-8") as poem_file:
                    poem=poem_file.read()
                    data.append(poem)
    return data

Here we'll use data originally released on Github to support "Literary Pattern Recognition": [https://github.com/hoytlong/PatternRecognition](https://github.com/hoytlong/PatternRecognition)

In [36]:
haiku=read_texts("../data/haiku/long_so_haiku", "../data/haiku/Haikus.csv", 4)

In [37]:
others=read_texts("../data/haiku/long_so_others", "../data/haiku/OthersData.csv", 5)

In [38]:
# don't change anything within this code block

def run_all(haiku, others, feature_function):
    
    X, Y, featurize_vocab=feature_function(haiku, others)
    kmeans = KMeans(n_clusters=2, random_state=0).fit(X)
    nmi=metrics.normalized_mutual_info_score(Y, kmeans.labels_)
    print("%.3f NMI" % nmi)

As one example, let's take a simple featurization and represent each poem by a binary indicator of the dictionary word types it contains.  "To be or not to be", for example, would be represented as {"to": 1, "be": 1, "or": 1, "not": 1}

In [39]:
# This function takes in a list of haiku poems and non-haiku poems, and returns:

# X (sparse matrix, with poems as rows and features as columns)
# Y (list of poem labels, with 1=haiku and 0=non-haiku)
# feature_vocab (dict mapping feature name to feature ID)

def unigram_featurize_all(haiku, others):

    def unigram_featurize(poem, feature_vocab):
        
        # featurize text by just noting the binary presence of words within it
        
        feats={}

        tokens=nltk.word_tokenize(poem.lower())
        for token in tokens:
            if token not in feature_vocab:
                feature_vocab[token]=len(feature_vocab)
            feats[feature_vocab[token]]=1
        return feats

    feature_vocab={}
    data=[]
    Y=[]

    for poem in haiku:
        feats=unigram_featurize(poem, feature_vocab)
        data.append(feats)
        Y.append(1)
    for poem in others:
        feats=unigram_featurize(poem, feature_vocab)
        data.append(feats)
        Y.append(0)
    
    # since the data above has all haiku ordered before non-haiku, let's shuffle them
    temp = list(zip(data, Y))
    random.shuffle(temp)
    data, Y = zip(*temp)

    # we'll use a sparse representation since our features are sparse
    X=sparse.lil_matrix((len(data), len(feature_vocab)))

    for idx,feats in enumerate(data):
        for f in feats:
            X[idx,f]=feats[f]
    
    return X, Y, feature_vocab

This method yields an NMI of ~0.07 (with some variability due to the randomness of KMeans)

In [40]:
run_all(haiku, others, unigram_featurize_all)

0.073 NMI


/Users/piadeshpande/anaconda3/envs/anlp/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


**Q1**: Copy the `unigram_featurize_all` code above and adapt it to create your own featurization method named `fancy_featurize_all`.  You may use whatever information you like to represent these poems for the purposes of clustering them into two categories, but you must use the KMeans clustering (with 2 clusters) as defined in `run_all`.  Use your own understanding of haiku, or read the Long and So article above for other ideas.  Are you able to improve over an NMI of 0.07?

In [41]:
def fancy_featurize_all(haiku, others):
    
    def syllable_count(poem):
        feats={}
        tokens=nltk.word_tokenize(poem.lower())
        for token in tokens:
           
            count = 0
            vowels = "aeiouy"
            if token[0] in vowels:
                count += 1
            for index in range(1, len(token)):
                if token[index] in vowels and token[index - 1] not in vowels:
                    count += 1
            if token.endswith("e"):
                count -= 1
            if count == 0:
                count += 1
            return count

    feature_vocab={}
    data=[]
    Y=[]

    for poem in haiku:
        feats=syllable_count(poem)
        data.append(feats)
        Y.append(1)
    for poem in others:
        feats=syllable_count(poem)
        data.append(feats)
        Y.append(0)
    
    # since the data above has all haiku ordered before non-haiku, let's shuffle them
    temp = list(zip(data, Y))
    random.shuffle(temp)
    data, Y = zip(*temp)

    # we'll use a sparse representation since our features are sparse
    X=sparse.lil_matrix((len(data), len(feature_vocab)))

    for idx,feats in enumerate(data):
        for f in feats:
            X[idx,f]=feats[f]
    
    return X, Y, feature_vocab

In [42]:
run_all(haiku, others, fancy_featurize_all)
# I am getting an error here that I can't figure out related to a poem not being iterable? 


TypeError: 'int' object is not iterable

**Q2**: Describe your method for featurization in 100 words and why you expect it to be able to separate haiku poems from non-haiku poems in this data.

My method for featurization counts the number of syllables in each poem. I believe this will separate haiku poems from non-haiku poems because haikus (normally) have 17 syllables (5+7+5) while non-haiku poems can have any number of syllables. 